<a href="https://colab.research.google.com/github/Pranavla/AI_PDF_Search_ChatBot/blob/main/AI_PDF_Search_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q langchain transformers sentence_transformers llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.0/466.0 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 KB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:

import PyPDF2
from llama_index import Document, GPTSimpleVectorIndex, ServiceContext
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTListIndex,GPTSimpleVectorIndex, PromptHelper
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LLMPredictor, ServiceContext
import torch
from langchain.llms.base import LLM
from transformers import pipeline


In [6]:
class customLLM(LLM):
    model_name = "google/flan-t5-large"
    pipeline = pipeline("text2text-generation", model=model_name, device=0, model_kwargs={"torch_dtype":torch.bfloat16})
    def _call(self, prompt, stop=None):
        return self.pipeline(prompt, max_length=9999)[0]["generated_text"]
    def _identifying_params(self):
        return {"name_of_model": self.model_name}
    def _llm_type(self):
        return "custom"
llm_predictor = LLMPredictor(llm=customLLM())

In [7]:
hfemb = HuggingFaceEmbeddings()
embed_model = LangchainEmbedding(hfemb)

In [10]:
import PyPDF2
from llama_index import Document, GPTSimpleVectorIndex, ServiceContext
# Load PDF document
pdf_file = open('/content/bitcoin.pdf', 'rb')
pdf_reader = PyPDF2.PdfReader(pdf_file)
pdf_text = ''
for page in pdf_reader.pages:
    pdf_text += page.extract_text()
document = Document(pdf_text)
document_list = [document]

In [11]:
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model=embed_model)
index = GPTSimpleVectorIndex.from_documents(document_list, service_context=service_context)

In [12]:
import logging
logging.getLogger().setLevel(logging.CRITICAL)

In [13]:
response = index.query( "What is the title of this document?") 

Token indices sequence length is longer than the specified maximum sequence length for this model (3118 > 512). Running this sequence through the model will result in indexing errors


In [14]:
response.response

'Bitcoin'

In [15]:
# temp fix for running shell commands on Google Colab

import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [16]:
!pip install gradio==2.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 70.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.2/211.2 KB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 KB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 KB 67.3 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4707 sha256=486c7bc147c75fed4f933cafe38dd3b8f199e2b4db5239fc23b59e51aeb4fed1
  Stored in directory: /root/.cache/pip/wheels/91/e2/96/f676aa08bfd789328c6576cd0f1fde4a3d686703bb0c247697
  Created wheel for flask-cachebuster: filename=Flask_CacheBuster-1.0.0-py3-none-any.whl size=3370 sha256=328adc60a0411e9dfdcb3cb8cc783937e213ad29eae5783139a8b0317ef

In [17]:
import gradio as gr

In [18]:
# Define a function to create and query the index based on a PDF document and a prompt
def search_pdf_with_prompt(file, prompt):
    # Load PDF document
    pdf_reader = PyPDF2.PdfReader(file)
    pdf_text = ''
    for page in pdf_reader.pages:
        pdf_text += page.extract_text()
    document = Document(pdf_text)
    document_list = [document]
    service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model=embed_model)
    index = GPTSimpleVectorIndex.from_documents(document_list, service_context=service_context)
    response = index.query(prompt) 
    return response

# Define the Gradio interface
inputs = [
    gr.inputs.File(label="PDF file"),
    gr.inputs.Textbox(label="Search prompt")
]
outputs = gr.outputs.Textbox(label="Search results")

gradio_app = gr.Interface(search_pdf_with_prompt, inputs, outputs, title="PDF Search", 
                          description="Enter a PDF file and a search prompt to index and search through its contents.")

# Run the interface
gradio_app.launch()



IMPORTANT: You are using gradio version 2.4.0, however version 3.14.0 is available, please upgrade.
--------
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
This share link will expire in 72 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://e87d5666eb46ae47.gradio.app


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7860/',
 'https://e87d5666eb46ae47.gradio.app')